<a href="https://colab.research.google.com/github/bhanuchaddha/The-Ai-Handbook/blob/main/4-ai-agents/customer_care_support/Multi_Agent_Customer_Care_Representative_With_CrewAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this lesson, you will learn about the six key elements which help make Agents perform even better:

* Role Playing
* Focus
Tools
Cooperation
Guardrails
Memory

By not setting allow_delegation=False, allow_delegation takes its default value of being True.
This means the agent can delegate its work to another agent which is better suited to do a particular task.

Role Playing: Both agents have been given a role, goal and backstory.
Focus: Both agents have been prompted to get into the character of the roles they are playing.
Cooperation: Support Quality Assurance Agent can delegate work back to the Support Agent, allowing for these agents to work together.




### Possible Custom Tools
- Load customer data
- Tap into previous conversations
- Load data from a CRM
- Checking existing bug reports
- Checking existing feature requests
- Checking ongoing tickets
- ... and more



- Some ways of using CrewAI tools.

```Python
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
```

In [1]:
pip install crewai crewai-tools


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.0/192.0 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.0/468.0 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.4/211.4 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━

In [3]:
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY']=userdata.get('OPENAI_API_KEY')

In [5]:
from crewai import Agent, Task, Crew

In [6]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at {employer_name} and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True
)

In [7]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI {employer_name} and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True
)

In [8]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

In [10]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [32]:
maersk_ocean_api_doc_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/quickstart"
)

##### Different Ways to Give Agents Tools

- Agent Level: The Agent can use the Tool(s) on any Task it performs.
- Task Level: The Agent will only use the Tool(s) when performing that specific Task.

**Note**: Task Tools override the Agent Tools.

In [33]:
inquiry_resolution_task = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[maersk_ocean_api_doc_tool],
    agent=support_agent,
)

In [34]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


### Creating the Crew

#### Memory
- Setting `memory=True` when putting the crew together enables Memory.

In [35]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution_task, quality_assurance_review],
  verbose=True,
  memory=True
)

In [36]:
inputs = {
    "employer_name": "CrewAI",
    "customer": "Maersk",
    "person": "Bhanu",
    "inquiry": "I need help with setting up a Crew AI "
               "Can you tell how to setup multiple agents which colaborate? "
               "Can you also provide how to use memory in agents?"
}
result = crew.kickoff(inputs=inputs)

# Agent: Senior Support Representative
## Task: Maersk just reached out with a super important ask:
I need help with setting up a Crew AI Can you tell how to setup multiple agents which colaborate? Can you also provide how to use memory in agents?

Bhanu from Maersk is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


# Agent: Senior Support Representative
## Thought: I need to gather information on how to set up multiple agents that collaborate and how to use memory in agents within CrewAI. This will provide Bhanu from Maersk a detailed, informative response to her inquiry.
## Using tool: Read website content
## Tool Input: 
"{}"
## Tool Output: 
Quickstart - CrewAI CrewAI home page Search CrewAI docs crewAIInc / crewAI crewAIInc / crewAI Search... Navigation Get Started Quickstart Get Started Examples CrewAI home page Community Changelog Get Start